In [1]:
import numpy as np
import random
from random import randint
import collections
from copy import deepcopy
from Chromosome_copy import *
import math

~~~~~~~~~
No duplicates found for Teachers
              Name
0      Ayesha Bano
1    Aqeel Shahzad
2        Farah Naz
3       Hamda Khan
4     Usman Rashid
..             ...
58    Kashif Munir
59   Hammad Majeed
60    Noreen Jamil
61  Hassan Mustafa
62    Farwa Batool

[63 rows x 1 columns]
~~~~~~~~~


~~~~~~~~~
Duplicates found for Courses. Removing them!
     Code                                      Title
14  CS328                       Software Engineering
16  EE229  Computer Organization & Assembly Language
17  CS211                        Discrete Structures
      Code                                        Title
0    CS217                  Object Oriented Programming
1    EE227                         Digital Logic Design
2    CS211                          Discrete Structures
3    SE110                Intro to Software Engineering
4    CS118                     Programming Fundamentals
5    CS219                             Database Systems
6    CS220                         

In [2]:
#---------------------------------------------------------------#
TOTAL_POPULATION = 100
MAX_GENERATIONS = 1000
CROSS_PROBABILITY = 0.8
MUTATION_PROBABILITY = 0.05
STUDENT_GROUPS_GLOBAL = {}
TEACHERS_GLOBAL = []
COURSES_GLOBAL = []
CLASS_IDS_GLOBAL = []
STUDENT_LIST_GLOBAL = []
#---------------------------------------------------------------#


In [9]:
def main():
    courses_code_list, student_groups, teachers_list, class_IDs, student_names_list = create_Datasets()
    global STUDENT_GROUPS_GLOBAL
    global TEACHERS_GLOBAL
    global COURSES_GLOBAL
    global CLASS_IDS_GLOBAL
    global STUDENT_LIST_GLOBAL
    STUDENT_GROUPS_GLOBAL = student_groups
    TEACHERS_GLOBAL = deepcopy(teachers_list)
    COURSES_GLOBAL = deepcopy(courses_code_list)
    CLASS_IDS_GLOBAL = deepcopy(class_IDs)
    STUDENT_LIST_GLOBAL = deepcopy(student_names_list)
    population = []
    for i in range(TOTAL_POPULATION):
        population.append(populate(courses_code_list,
                                   student_groups, teachers_list, class_IDs))
        population[i].index = i
        population[i].chromo_To_Bits(TOTAL_POPULATION)
        print("CHROMO {} = {}\n\n".format(i, population[i].chromoBits))
        if i == 99:
            print("Teachers Size: ", len(
                population[i].days[0].sessions[0].teachers) + len(population[i].days[0].sessions[1].teachers))
    
    GA(population)

In [13]:
# def calculate_fitness(population):
#     # print("Fitness Calc")
#     ### Your Code Here ####
#     global STUDENT_GROUPS_GLOBAL
#     global TEACHERS_GLOBAL
#     global COURSES_GLOBAL
#     global STUDENT_LIST_GLOBAL
#     # fitness_values = []

#     # Value of fitness in terms of exams per day
#     # relativity = [0.1, 0.1, 0.5, 0.75, 0.8, 1, 1, 1, 0.8, 0.5, 0.5]
#     # relativity = [0.07, 0.5, 0.5, 0.6, 0.6,
#     #               0.4, 0.3, 0.001, 0.001, 0.001, 0.001]

#     # -> Overlapping Students, Overlapping Teachers, Consecutive Teachers, Duplicate Exams
#     # -> 100                   10                    10                    100

#     for i, chromo in enumerate(population):
#         overlap_student = 0
#         overlap_teacher = 0
#         consecutive_Students = 0
#         consecutive_Teachers = 0
#         population[i].remove_Empty_Days()
#         FITNESS_CONSTANT = 100000
#         # total_Conflicts = 0
#         for day in chromo.days:
#             for session in day.sessions:

#                 overlap = session.overlapping_Students1()
#                 if overlap == True:
                    
#                     overlap_student += 2

#                 overlap = session.overlapping_Teachers()
#                 if overlap > 0:
                    
#                     overlap_teacher += overlap
#             overlap = day.consecutive_Students()
#             if overlap > 0:
#                 consecutive_Students += 2
                
#             overlap = day.consecutive_Teachers()
#             if overlap > 0:
#                 consecutive_Teachers += 5
                

#         duplicate_Exams = chromo.duplicate_Exams()
#         total_Exams = chromo.total_exams()
#         total_Days = chromo.days_count
#         remaining_Exams = len(COURSES_GLOBAL) - total_Exams + duplicate_Exams
        
#         if overlap_student > 0:
#             FITNESS_CONSTANT -= overlap_student*40
            
#         if overlap_teacher > 0:
#             FITNESS_CONSTANT -= 10
#         # if consecutive_Students > 0:
#             # FITNESS_CONSTANT -= consecutive_Students
#         if consecutive_Teachers > 0:
#             FITNESS_CONSTANT -= 10
#         if duplicate_Exams > 0:
#             FITNESS_CONSTANT -= 20*duplicate_Exams
            

#         if remaining_Exams > 0:
#             FITNESS_CONSTANT -= 20*remaining_Exams
            
#         if total_Days >= len(COURSES_GLOBAL)/2:
#             FITNESS_CONSTANT -= total_Days*5
            
#         population[i].fitness = FITNESS_CONSTANT
#         # print("Chromo {} with fitness {}".format(i, FITNESS_CONSTANT))
#     for i, chromo in enumerate(population):
#         population[i].index = i
#     return population

def calculate_fitness(population):
    # print("Fitness Calc")
    ### Your Code Here ####
    global STUDENT_GROUPS_GLOBAL
    global TEACHERS_GLOBAL
    global COURSES_GLOBAL
    global STUDENT_LIST_GLOBAL
    # fitness_values = []

    # Value of fitness in terms of exams per day
    # relativity = [0.1, 0.1, 0.5, 0.75, 0.8, 1, 1, 1, 0.8, 0.5, 0.5]
    # relativity = [0.07, 0.5, 0.5, 0.6, 0.6,
    #               0.4, 0.3, 0.001, 0.001, 0.001, 0.001]

    # -> Overlapping Students, Overlapping Teachers, Consecutive Teachers, Duplicate Exams
    # -> 100                   10                    10                    100

    for i, chromo in enumerate(population):
        overlap_student = 0
        overlap_teacher = 0
        consecutive_Students = 0
        consecutive_Teachers = 0
        weekend_days = 0
        population[i].remove_Empty_Days()
        chromo.remove_Empty_Days()
        FITNESS_CONSTANT = 100000
        # total_Conflicts = 0
        for j, day in enumerate(chromo.days):
            for session in day.sessions:

                overlap = session.overlapping_Students1()
                # print("Students overlap:", overlap)
                if overlap == True:
                    overlap_student += 2

                overlap = session.overlapping_Teachers()
                if overlap > 0:
                    # print("T")
                    overlap_teacher += overlap
                if j % 5 == 0 or j % 6 == 0:
                    if session.courseIDs is not None:
                        weekend_days += 2
            overlap = day.consecutive_Students()
            if overlap > 0:
                consecutive_Students += 2

            overlap = day.consecutive_Teachers()
            if overlap > 0:
                consecutive_Teachers += 5

        duplicate_Exams = chromo.duplicate_Exams()
        total_Exams = chromo.total_exams()
        total_Days = chromo.days_count
        remaining_Exams = len(COURSES_GLOBAL) - total_Exams + duplicate_Exams
        # print("Rems: ", remaining_Exams)
        if overlap_student > 0:

            FITNESS_CONSTANT -= overlap_student*40

        if overlap_teacher > 0:
            # print("ttt")
            FITNESS_CONSTANT -= 10

        if weekend_days > 0:
            FITNESS_CONSTANT -= weekend_days*20
        # if consecutive_Students > 0:
            # FITNESS_CONSTANT -= consecutive_Students
        if consecutive_Teachers > 0:
            # print(" c t")
            FITNESS_CONSTANT -= 10
        if duplicate_Exams > 0:
            # print("dup")
            FITNESS_CONSTANT -= 20*duplicate_Exams

        if remaining_Exams > 0:
            # print("r")
            FITNESS_CONSTANT -= 20*remaining_Exams

        # if total_Days >= len(COURSES_GLOBAL)/2:
        #     FITNESS_CONSTANT -= total_Days*5

        population[i].fitness = FITNESS_CONSTANT
        # print("Chromo {} with fitness {}".format(i, FITNESS_CONSTANT))
    for i, chromo in enumerate(population):
        population[i].index = i
    return population

In [4]:
def roulette_wheel_selection(population):
    # print("Wheel Sel")

    # Sum of fitnesses of population
    population_fitness = sum(
        [chromosome.fitness for chromosome in population])
    # pick = random.uniform(0, population_fitness)
    # Probability = Chromo / (sum of fitnesses)
    chromosome_probabilities = [chromosome.fitness /
                                population_fitness for chromosome in population]

    # Selects one chromosome randomly given the probabilities
    chromosome = np.random.choice(population, p=chromosome_probabilities)

    # range_list = []
    # range_list.append(0.0)
    # for prob in chromosome_probabilities:
    #     range_list.append(float(range_list[-1]+prob))
    # rand_num = random.random()
    # for i, value in enumerate(range_list):

    #     if range_list[i-1] < rand_num and rand_num <= value:
    #         return population[i-1]

    # chromosome = population[3]
    # print("The Chromo Selected has fitness: ", chromosome.fitness)
    return chromosome

In [5]:
def cross_Over(population, list_best):
    # print("Crossover")
    crossed_population = []
    global STUDENT_GROUPS_GLOBAL
    global COURSES_GLOBAL
    # dimensions = ["Day", "Session", "Student Group", "Teacher"]
    while len(crossed_population) < len(population):
        random_b = randint(0, len(population)-1)
        parent_b = population[random_b]
        parent_a = roulette_wheel_selection(population)

        if random.random() <= CROSS_PROBABILITY:
            pointer = randint(1, min(len(parent_a.days), len(parent_b.days)))

            for shift in range(pointer):
                parent_a.days[shift], parent_b.days[shift] = parent_b.days[shift], parent_a.days[shift]

            if len(crossed_population) < len(population):
                crossed_population.append(parent_a)
            if len(crossed_population) < len(population):
                crossed_population.append(parent_b)
        else:
            if len(list_best) > 0:
                if len(crossed_population) < len(population):
                    crossed_population.append(
                        list_best[randint(0, len(list_best)-1)])
            elif len(crossed_population) < len(population):
                crossed_population.append(parent_a)
        # print("DEEE Random In")
    for i in range(len(crossed_population)):
        crossed_population[i].index = i

    return crossed_population

In [6]:
def mutate(population):

    # mutate_population = []
    global COURSES_GLOBAL
    global STUDENT_GROUPS_GLOBAL
    global TEACHERS_GLOBAL
    global CLASS_IDS_GLOBAL

    for chromo in population:
        if random.random() <= MUTATION_PROBABILITY:
            new_chromo = populate(
                COURSES_GLOBAL, STUDENT_GROUPS_GLOBAL, TEACHERS_GLOBAL, CLASS_IDS_GLOBAL)
            random_day1 = randint(0, new_chromo.days_count-1)
            random_day2 = randint(0, chromo.days_count-1)
            # print("day1:", random_day1)
            # print("day2:", random_day2)
            # print("mutation")
            population[chromo.index].days[random_day2] = new_chromo.days[random_day1]

    return population

In [7]:
def best_Schedule(population):
    # print("Best Schedule")
    best = 0
    chromos = []
    index = 0
    for i, chromo in enumerate(population):
        if chromo.fitness >= best:
            best = chromo.fitness
            index = i
    best_chromo = None
    for chromo in population:
        if chromo.fitness == best:
            chromos.append(chromo)
            if best_chromo == None:
                best_chromo = chromo

    return best_chromo, chromos

In [8]:

def GA(population):
    print("MAX GENERATIONS: ", MAX_GENERATIONS)
    best_Chromo = None
    list_best = []
    for i in range(MAX_GENERATIONS):

        population = deepcopy(calculate_fitness(population))

        population = deepcopy(cross_Over(population, list_best))

        population = deepcopy(mutate(population))

        population = deepcopy(calculate_fitness(population))
        temp_best, list_best_temp = best_Schedule(population)
        if best_Chromo == None:
            best_Chromo = temp_best
            list_best = list_best_temp
        if best_Chromo.fitness < temp_best.fitness:
            best_Chromo = temp_best
            list_best = list_best_temp
        # if best_Chromo.fitness < temp_best1.fitness:
        #     best_Chromo = temp_best1
        if i % 50 == 0:
            print("{} iterations passed".format(i))
            print("BEST SOLUTION AFTER {} ITERATIONS: ".format(
                i), best_Chromo.fitness)
            best_courses_list = []
            for j, day in enumerate(best_Chromo.days):
                print("Day {}...\nSession 1:".format(j+1))
                print(day.sessions[0].courseIDs)
                best_courses_list.append(day.sessions[0].courseIDs)
                best_courses_list.append(day.sessions[1].courseIDs)
                print(day.sessions[0].teachers)
                print("Student_Overlap: ",
                      day.sessions[0].overlapping_Students())
                print("Session 2:")
                print(day.sessions[1].courseIDs)
                print(day.sessions[1].teachers)
                print("Student_Overlap: ",
                      day.sessions[1].overlapping_Students())

            best_courses = []
            for list_exam in best_courses_list:
                for exam in list_exam:
                    best_courses.append(exam)
            print("Courses Remaining: ", set(
                COURSES_GLOBAL) - set(best_courses))
            print("Best Courses: ", sorted(best_courses))
            print("Total: ", best_Chromo.total_exams())

    print("BEST SOLUTION FOUND WITH FITNESS: ", best_Chromo.fitness)
    print("And duplicate exams: ", best_Chromo.duplicate_Exams())
    best_courses_list = []
    for i, day in enumerate(best_Chromo.days):
        print("Day {}...\nSession 1:".format(i+1))
        print(day.sessions[0].courseIDs)
        best_courses_list.append(day.sessions[0].courseIDs)
        best_courses_list.append(day.sessions[1].courseIDs)
        print(day.sessions[0].teachers)
        print("Student_Overlap: ", day.sessions[0].overlapping_Students())
        print("Session 2:")
        print(day.sessions[1].courseIDs)
        print(day.sessions[1].teachers)
        print("Student_Overlap: ", day.sessions[1].overlapping_Students())

    best_courses = []
    for list_exam in best_courses_list:
        for exam in list_exam:
            best_courses.append(exam)
    print("Courses Remaining: ", set(COURSES_GLOBAL) - set(best_courses))
    print("Best Courses: ", sorted(best_courses))
    return best_Chromo


In [11]:
main()

t'}
Best Courses:  ['CG', 'CS', 'Drawing', 'English', 'History', 'Math', 'OOP', 'PF', 'Physics', 'Physics2', 'Physics3', 'Science']
Total:  12
150 iterations passed
BEST SOLUTION AFTER 150 ITERATIONS:  99945
Day 1...
Session 1:
['English']
['Shehreyar Rashid']
Student_Overlap:  0
Session 2:
['Physics']
['Shams Farooq']
Student_Overlap:  0
Day 2...
Session 1:
['CS']
['Behjat Zuhaira']
Student_Overlap:  0
Session 2:
['History']
['Adnan Tariq']
Student_Overlap:  0
Day 3...
Session 1:
['Physics2']
['Usman Rashid']
Student_Overlap:  0
Session 2:
['Math']
['Tayyab Nadeem']
Student_Overlap:  0
Day 4...
Session 1:
['Physics3']
['Shoaib Mehboob']
Student_Overlap:  0
Session 2:
['OOP']
['Mehreen Alam']
Student_Overlap:  0
Day 5...
Session 1:
['PF']
['Hassan Raza']
Student_Overlap:  0
Session 2:
['Drawing']
['Hassan Mustafa']
Student_Overlap:  0
Day 6...
Session 1:
['Science']
['Muhammad bin Qasim']
Student_Overlap:  0
Session 2:
['CG']
['Tayyab Nadeem']
Student_Overlap:  0
Day 7...
Session 1:
[]

In [16]:
main()

0000111010110101010100100000000111001000010010011000100000001001001100110111111001010000000011010100010010011110010000000101001011110111010000011000000001110111111000101001000000000010110011010101100001


CHROMO 79 = 1001111000000011010110001100101000000011100100000100000010100101000110101000100000000111110000011000001000110110001001001101100000001100001011000000011011100111000001110000000000011010111010000001000100110100000001


CHROMO 80 = 101000000001011011100000001000100001100000000001001011110111111010011010100000001010101000110111010101101111000000000000010010100000010101101110110000000011001110001001110100100100000001110100110100000


CHROMO 81 = 101000100010100010111100001100001100010000000001001011101101010110010010100000001010011100011000101001011110001010000000010010111011111011100011001000000011001100001100000110101100000001100000000110000


CHROMO 82 = 1010010000001000101001001001101010001010101011001011101100001100101101001101101110001


CHROMO 83 = 1010011000010111010010

In [ ]:
main()